In [22]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import networkx as nx
import funcy as fy
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt

import usgoc.preprocessing.graph.wl2 as wl2
import usgoc.datasets.unsafe_go as dataset
import usgoc.models.gnn as gnn
import usgoc.utils as utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
split_i = 0 # the evaluated split number (between 0-9)
in_enc = "wl1"

with utils.cache_env(use_cache=True):
  ds = dataset.load_dataset()
  splits = dataset.get_split_idxs(ds)

In [3]:
from transformers import CodeGenTokenizer

checkpoint = "Salesforce/codegen-350M-multi"
tokenizer = CodeGenTokenizer.from_pretrained(checkpoint, cache_dir="/app/.cache")
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
tokenizer

Using pad_token, but it is not set yet.


PreTrainedTokenizer(name_or_path='Salesforce/codegen-350M-multi', vocab_size=50257, model_max_len=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|pad|>'})

In [13]:
def get_block_snippets(g, snippets=None):
  if snippets is None:
    snippets = set()
  for node, data in g.nodes(data=True):
    if "code" in data:
      snippets.add(data["code"])
  return snippets

graphs = ds[0]
snippets = {g.source_code for g in graphs}

for g in graphs:
  get_block_snippets(g, snippets)

snippets = list(snippets)

import json

with open("/app/data/unsafe-go-dataset/snippets.json", "w") as f:
  json.dump(snippets, f)

len(snippets)

8803

In [24]:
import pickle
import json

with open("/app/data/unsafe-go-dataset/embs.pickle", "rb") as f:
  embs = pickle.load(f)
  
embs = [e.reshape((-1, 1024)).mean(axis=0) for e in embs]

with open("/app/data/unsafe-go-dataset/snippets.json", "r") as f:
  snippets = json.load(f)

In [27]:
emb_dict = fy.zipdict(snippets, embs)

with open("/app/data/unsafe-go-dataset/emb_dict.pickle", "wb") as f:
  pickle.dump(emb_dict, f)

In [29]:
with open("/app/data/unsafe-go-dataset/emb_dict.pickle", "rb") as f:
  emb_dict = pickle.load(f)